In [163]:
import pandas as pd
import xlrd

## Load TPM data matrix

In [164]:
cancer_tpm = pd.read_csv('data/GSE131512_cancerTPM.txt', sep='\t')

In [165]:
cancer_tpm.head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C87,C88,C89,C90,C91,C92,C93,C94,C95,C96
ENSG00000000003,7.071605,60.610797,58.255903,29.917356,24.500322,25.311091,37.394452,15.354658,24.839014,24.591295,...,14.134044,10.803910,6.068982,12.714350,15.983392,0.000000,38.189128,15.608352,51.519267,30.805593
ENSG00000000005,13.279391,47.424080,60.455497,6.482332,53.675826,31.686900,32.409716,0.000000,48.976056,46.178684,...,0.000000,12.172853,136.759411,28.650721,0.000000,0.000000,5.976111,6.280734,241.863225,0.000000
ENSG00000000419,0.000000,0.000000,23.040206,8.646690,20.456404,42.266702,21.615428,11.538256,18.665274,76.996356,...,79.657691,8.118597,0.000000,0.000000,0.000000,179.178412,15.942897,0.000000,0.000000,0.000000
ENSG00000000457,6.212355,2.218590,12.120963,6.065120,16.142519,37.059356,7.580945,0.000000,0.000000,21.603278,...,18.624986,15.660412,33.988677,0.000000,36.858404,83.788404,18.172309,2.938248,14.547636,10.148436
ENSG00000000460,34.038592,35.828348,55.926653,10.494273,14.482668,19.236768,13.117055,4.667898,18.877984,12.459809,...,17.903438,19.706680,23.062537,1.932615,28.344382,3.020339,27.411791,1.694651,3.729079,7.804221


In [166]:
cancer_tpm.index = new_indices
cancer_tpm.to_csv('data/cancer_tpm_updated_indices.tsv', sep='\t', header=True)

In [167]:
cancer_tpm = pd.read_csv('data/cancer_tpm_updated_indices.tsv', sep='\t', index_col=0).transpose()

## Load metadata
#### Sample ID: C1-C28, recurrence samples; C29-96, non-recurrence samples

In [168]:
wb = xlrd.open_workbook('data/GSE131512_metaData.xlsx') 
sheet = wb.sheet_by_index(0) 

In [169]:
metadata = []
for i in range(1, sheet.nrows): 
    metadata.append(sheet.row_values(i)) 

metadata_df = pd.DataFrame(metadata, columns=sheet.row_values(0))
metadata_df = metadata_df[metadata_df['Sample name'].str.contains('C')]

def add_recurrence_information(r):
    sample_number = int(r['Sample name'].split('C')[1])
    if sample_number >= 1 and sample_number <= 28:
        return 1
    elif sample_number >= 29:
        return 0

metadata_df['recurrence'] = metadata_df.apply(add_recurrence_information, axis=1)
metadata_df.head()

,Sample name,Age,Gender,recurrence
0,C1,34.443836,Female,1
1,C2,43.353425,Female,1
2,C3,43.852055,Female,1
3,C4,41.421918,Female,1
4,C5,42.115068,Female,1


In [170]:
metadata_df.tail()

,Sample name,Age,Gender,recurrence
91,C92,40.767123,Female,0
92,C93,43.443836,Female,0
93,C94,43.904110,Female,0
94,C95,39.454795,Female,0
95,C96,39.767123,Female,0


## Add recurrence information to TPM table as a multiindex

In [171]:
multi_index_lists = [list(metadata_df['Sample name']), list(metadata_df['recurrence'])]
multi_index_tuples = list(zip(*multi_index_lists))

In [172]:
cancer_tpm.index = pd.MultiIndex.from_tuples(multi_index_tuples, names=['sample', 'recurrence'])

In [173]:
cancer_tpm.to_csv('data/cancer_tpm_multiindexed.tsv', sep='\t', header=True)

## Example of how to read in the csv with multiindex

In [174]:
cancer_tpm = pd.read_csv('data/cancer_tpm_multiindexed.tsv', sep='\t', index_col=[0,1])

In [175]:
cancer_tpm.head()

,,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AL132857.2,BMS1P21,AC010642.2,AC011451.3,AC141257.4,AC004949.1,AC009951.5,HYMAI,RARRES2P11,AC022726.2
sample,recurrence,,,,,,,,,,,,,,,,,,,,,
C1,1,7.071605,13.279391,0.000000,6.212355,34.038592,9.231355,32.811263,19.136966,12.631348,11.220057,...,12.308474,0.0,4.430132,0.000000,0.0,6.856902,16.599238,22.357146,0.0,19.542796
C2,1,60.610797,47.424080,0.000000,2.218590,35.828348,4.395669,22.498053,0.000000,5.413170,48.083612,...,8.791338,0.0,0.000000,48.996429,0.0,9.795095,0.000000,4.562460,0.0,0.000000
C3,1,58.255903,60.455497,23.040206,12.120963,55.926653,8.005046,13.657227,14.935300,16.430065,14.594347,...,0.000000,0.0,0.000000,14.871406,0.0,13.378542,0.000000,12.463189,0.0,0.000000
C4,1,29.917356,6.482332,8.646690,6.065120,10.494273,0.000000,23.064209,18.683413,9.865584,8.215603,...,12.016758,0.0,0.000000,33.486271,0.0,13.388781,0.000000,6.236364,0.0,0.000000
C5,1,24.500322,53.675826,20.456404,16.142519,14.482668,0.000000,59.112604,8.840272,55.432674,0.000000,...,0.000000,0.0,0.000000,26.407358,0.0,19.797049,57.509813,3.688509,0.0,45.138720


### Example of how to select using multiindex

In [176]:
cancer_tpm.query('recurrence == 0').head()

,,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AL132857.2,BMS1P21,AC010642.2,AC011451.3,AC141257.4,AC004949.1,AC009951.5,HYMAI,RARRES2P11,AC022726.2
sample,recurrence,,,,,,,,,,,,,,,,,,,,,
C29,0,3.900178,0.000000,0.000000,2.569709,2.964188,66.187389,10.857770,0.000000,0.000000,4.641120,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000
C30,0,69.498909,279.660651,0.000000,0.000000,5.030486,4.320223,47.909126,26.867982,21.281042,7.876387,...,0.000000,0.0,0.0,0.000000,0.0,0.000000,46.610109,4.484151,0.0,0.000000
C31,0,12.846356,0.000000,28.960178,5.078445,33.195570,3.353957,25.749491,4.171731,44.056793,3.057372,...,6.707913,0.0,0.0,12.461652,0.0,33.632073,0.000000,17.406102,0.0,21.300996
C32,0,7.294690,30.821197,41.111953,38.449999,8.316093,0.000000,38.584834,11.844403,9.772367,0.000000,...,9.522573,0.0,0.0,53.071794,0.0,10.609820,462.317958,0.000000,0.0,120.955826
C33,0,7.405749,20.860294,9.275082,9.758847,11.256937,3.222517,13.744658,12.024731,9.259739,20.562889,...,25.780136,0.0,0.0,23.946576,0.0,0.000000,17.383578,13.379174,0.0,0.000000


In [177]:
cancer_tpm.query('recurrence == 1').head()

,,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AL132857.2,BMS1P21,AC010642.2,AC011451.3,AC141257.4,AC004949.1,AC009951.5,HYMAI,RARRES2P11,AC022726.2
sample,recurrence,,,,,,,,,,,,,,,,,,,,,
C1,1,7.071605,13.279391,0.000000,6.212355,34.038592,9.231355,32.811263,19.136966,12.631348,11.220057,...,12.308474,0.0,4.430132,0.000000,0.0,6.856902,16.599238,22.357146,0.0,19.542796
C2,1,60.610797,47.424080,0.000000,2.218590,35.828348,4.395669,22.498053,0.000000,5.413170,48.083612,...,8.791338,0.0,0.000000,48.996429,0.0,9.795095,0.000000,4.562460,0.0,0.000000
C3,1,58.255903,60.455497,23.040206,12.120963,55.926653,8.005046,13.657227,14.935300,16.430065,14.594347,...,0.000000,0.0,0.000000,14.871406,0.0,13.378542,0.000000,12.463189,0.0,0.000000
C4,1,29.917356,6.482332,8.646690,6.065120,10.494273,0.000000,23.064209,18.683413,9.865584,8.215603,...,12.016758,0.0,0.000000,33.486271,0.0,13.388781,0.000000,6.236364,0.0,0.000000
C5,1,24.500322,53.675826,20.456404,16.142519,14.482668,0.000000,59.112604,8.840272,55.432674,0.000000,...,0.000000,0.0,0.000000,26.407358,0.0,19.797049,57.509813,3.688509,0.0,45.138720


In [178]:
cancer_tpm.query('sample in ["C1", "C2", "C3"]').head()

,,TSPAN6,TNMD,DPM1,SCYL3,C1orf112,FGR,CFH,FUCA2,GCLC,NFYA,...,AL132857.2,BMS1P21,AC010642.2,AC011451.3,AC141257.4,AC004949.1,AC009951.5,HYMAI,RARRES2P11,AC022726.2
sample,recurrence,,,,,,,,,,,,,,,,,,,,,
C1,1,7.071605,13.279391,0.000000,6.212355,34.038592,9.231355,32.811263,19.136966,12.631348,11.220057,...,12.308474,0.0,4.430132,0.000000,0.0,6.856902,16.599238,22.357146,0.0,19.542796
C2,1,60.610797,47.424080,0.000000,2.218590,35.828348,4.395669,22.498053,0.000000,5.413170,48.083612,...,8.791338,0.0,0.000000,48.996429,0.0,9.795095,0.000000,4.562460,0.0,0.000000
C3,1,58.255903,60.455497,23.040206,12.120963,55.926653,8.005046,13.657227,14.935300,16.430065,14.594347,...,0.000000,0.0,0.000000,14.871406,0.0,13.378542,0.000000,12.463189,0.0,0.000000
